<a href="https://colab.research.google.com/github/aritapia19/coder-DataScience77695/blob/main/ProyectoDS_ParteI_Tapia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract

El presente trabajo realiza un análisis exploratorio de datos (EDA) sobre un dataset proveniente de un **programa de plan canje de equipos**. El objetivo es comprender el comportamiento de los procesos logísticos asociados a los retiros de equipos, tanto en **domicilio** como en **tienda**, identificando patrones, anomalías y oportunidades de mejora en los tiempos y en el cumplimiento de los SLA.

Se presentan las distribuciones de variables clave, la calidad de los datos, la relación entre distintas métricas de tiempos, y se sientan las bases para futuras etapas del proyecto, como modelado predictivo o análisis de optimización.

## Abstract — Resumen ejecutivo

Este notebook presenta un análisis exploratorio del dataset **"Plan Canje"**, que contiene registros de canje de equipos y las operaciones de retiro (a domicilio o en tienda). El objetivo principal es caracterizar los tiempos de proceso y el cumplimiento de los SLA (nivel de servicio), identificar factores asociados a retrasos o fallas en la entrega y preparar variables para modelado predictivo que permitan reducir demoras y mejorar la experiencia del cliente.

Se incluyen pasos de limpieza, análisis descriptivo, detección de valores faltantes, visualizaciones y análisis de correlaciones. Los resultados servirán como base para hipótesis comprobables y para proponer acciones operativas (p. ej. priorizar tipos de retiro o reforzar proveedores con más incumplimientos).

En lenguaje llano: buscamos entender qué hace que un equipo tarde más en llegar —y cómo evitarlo— sin perder la cordura en el intento.

## Preguntas de interés y hipótesis (testables)

A continuación se listan las preguntas que guían este trabajo y las hipótesis que proponemos verificar con el dataset.

1. **¿Los retiros a domicilio presentan mayores tiempos de entrega que los retiros en tienda?**

   **Hipótesis H1:** El tiempo total desde la creación del pedido hasta la entrega es mayor para retiros a domicilio que para retiros en tienda. (Test sugerido: prueba de medias / Mann–Whitney si no normal.)

2. **¿El proveedor/transportista y la zona geográfica influyen en el cumplimiento del SLA?**

   **Hipótesis H2:** Existen diferencias significativas en la tasa de cumplimiento del SLA entre distintos proveedores y por zonas/ciudades. (Test sugerido: chi-cuadrado sobre tablas de contingencia / análisis por grupos.)

3. **¿La falta de datos clave (dirección, DNI) aumenta la probabilidad de retraso o no entrega?**

   **Hipótesis H3:** Registros con campos faltantes o inválidos presentan mayor probabilidad de incumplimiento o tiempos de procesamiento más largos.

4. **¿Hay estacionalidad (día de la semana / mes) que afecte los tiempos y la tasa de cumplimiento?**

   **Hipótesis H4:** Pedidos realizados en fines de semana o en picos mensuales muestran mayor tiempo medio de entrega y mayor tasa de reprogramación.

5. **¿Algunos modelos/marcas de equipos generan más incidencias (devoluciones, rechazos, reprogramaciones)?**

   **Hipótesis H5:** Ciertos modelos o fabricantes presentan tasas más altas de incidencias operativas que impactan el SLA.

6. **¿Un indicador temprano (por ejemplo, tiempo transcurrido hasta el primer intento de salida) predice incumplimientos posteriores?**

   **Hipótesis H6:** Un mayor tiempo inicial entre creación y primer movimiento incrementa la probabilidad de incumplimiento final del SLA. (Modelado sugerido: regresión logística.)

---

**Variables recomendadas a construir / usar:** `tipo_retiro` (domicilio/tienda), `fecha_creacion`, `fecha_entrega`, `tiempo_total_dias`, `estado_final`, `vendor`, `dni_missing`, `direccion_missing`, `day_of_week`, `month`, `model_name`.

**Notas metodológicas rápidas:** para comparar tiempos use resumenes por grupo (media, mediana, IQR) y tests estadísticos adecuados; para predecir incumplimiento usar clasificación supervisada (logistic regression / tree-based) con validación cruzada y métricas claramante definidas (AUC, recall sobre incumplimientos).



# Preguntas de interés e hipótesis

**Preguntas de interés:**
- ¿Existe diferencia en los tiempos de retiro entre la modalidad *domicilio* y la modalidad *tienda*?
- ¿Qué variables impactan más en el cumplimiento de los SLA de despacho y visitas?
- ¿Hay patrones de localidades o provincias donde los retrasos sean sistemáticos?
- ¿Qué rol cumplen las fechas de visitas y sus motivos en el estado final del retiro?

**Hipótesis de trabajo:**
1. Los retiros en domicilio presentan mayor variabilidad en los tiempos y menor cumplimiento de SLA que los retiros en tienda.
2. El número de visitas y los motivos de no concreción son factores críticos en el cumplimiento E2E del proceso.
3. Ciertas provincias presentan sistemáticamente mayores tiempos de despacho, posiblemente por limitaciones logísticas.
4. La correcta planificación de fechas tentativas de despacho mejora el SLA global del plan canje.

# Proyecto DS - Parte I

Análisis exploratorio de datos (EDA) sobre dataset de **plan canje de equipos**, con foco en los retiros a domicilio o en tienda.

Este notebook documenta el proceso paso a paso, incluyendo carga de datos, limpieza inicial, exploración de variables, valores nulos, tipos de datos, estadísticas descriptivas, correlaciones y visualizaciones.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns

# Configuración visual
plt.style.use('default')
pd.set_option('display.max_columns', 50)


## 1. Carga de datos

In [ ]:
# Cargar dataset
csv_path = "Dataset plan canje.csv"
df = pd.read_csv(csv_path, sep=';', encoding='latin1')

# Normalizar nombres de columnas
df.columns = [c.strip().replace(' ', '_').lower() for c in df.columns]

df.shape, df.head()

## 2. Información general del dataset

In [3]:
df.info()

NameError: name 'df' is not defined

In [ ]:
df.dtypes.value_counts()

## 3. Análisis de valores nulos

In [ ]:
nulos = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_pct': (df.isna().mean()*100).round(2)
}).sort_values('missing_pct', ascending=False)
nulos.head(20)

## 4. Estadísticas descriptivas

In [ ]:
df.describe(include='all').transpose()

## 5. Variables numéricas y categóricas

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
num_cols, cat_cols

## 6. Visualizaciones univariadas y multivariadas

In [ ]:
# Histogramas y boxplots para variables numéricas
for col in num_cols:
    fig, ax = plt.subplots(1,2, figsize=(10,3))
    df[col].dropna().hist(bins=30, ax=ax[0])
    ax[0].set_title(f'Histograma - {col}')
    ax[1].boxplot(df[col].dropna(), vert=False)
    ax[1].set_title(f'Boxplot - {col}')
    plt.show()

In [ ]:
# Barras para variables categóricas (top 15 valores)
for col in cat_cols[:6]:  # mostrar solo primeras 6 por legibilidad
    vc = df[col].value_counts().nlargest(15)
    vc.plot(kind='barh', figsize=(8,4), title=f'Top valores - {col}')
    plt.show()

In [ ]:
# Scatter matrix (primeras numéricas)
num_for_scatter = num_cols[:6]
if len(num_for_scatter) >= 2:
    scatter_matrix(df[num_for_scatter].dropna(), figsize=(10,10), diagonal='hist')
    plt.show()

In [ ]:
# Matriz de correlación
if len(num_cols) >= 2:
    corr = df[num_cols].corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr, annot=False, cmap='coolwarm')
    plt.title('Matriz de correlación')
    plt.show()


In [ ]:
# Pares con mayor correlación absoluta
if len(num_cols) >= 2:
    corr = df[num_cols].corr()
    corr_abs = corr.abs().where(~np.eye(corr.shape[0],dtype=bool))
    stacked = corr_abs.unstack().dropna().sort_values(ascending=False)
    top_pairs = stacked[~stacked.index.duplicated()].nlargest(3)
    top_pairs

In [ ]:
# Graficar los 3 pares con mayor correlación
for (a,b), val in top_pairs.items():
    plt.scatter(df[a], df[b])
    plt.xlabel(a); plt.ylabel(b)
    plt.title(f'Scatter {a} vs {b} (corr={val:.2f})')
    plt.show()

## 7. Conclusiones

Este análisis exploratorio permitió:
- Identificar las variables numéricas y categóricas relevantes.
- Detectar presencia de valores nulos y necesidad de imputación o limpieza.
- Visualizar distribuciones de variables clave del proceso de plan canje.
- Analizar correlaciones entre métricas de tiempos y SLA.

Esto servirá de base para los siguientes pasos del proyecto: modelado de procesos, métricas de SLA, y propuestas de optimización para los retiros tanto a domicilio como en tienda.